In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, urlunparse
import os

# Starting URL
start_url = 'https://developer.community.com/reference/getting-started-with-your-api'
base_url = 'https://developer.community.com/reference/'

# Directory to save HTML files
save_dir = 'scraped_html'
os.makedirs(save_dir, exist_ok=True)

# Function to save HTML content to a file
def save_html(url, content):
    filename = os.path.join(save_dir, url.replace(base_url, '').replace('/', '_') + '.html')
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(content)

# Function to remove the fragment part of the URL
def remove_fragment(url):
    parsed_url = urlparse(url)
    return urlunparse(parsed_url._replace(fragment=''))

# Function to scrape a URL
def scrape(url, visited):
    url = remove_fragment(url)
    if url in visited:
        return
    visited.add(url)
    
    response = requests.get(url)
    if response.status_code == 200:
        save_html(url, response.text)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all subsequent links
        for link in soup.find_all('a', href=True):
            full_url = urljoin(base_url, link['href'])
            if full_url.startswith(base_url):
                scrape(remove_fragment(full_url), visited)

# Set to keep track of visited URLs
visited_urls = set()

# Start scraping
scrape(start_url, visited_urls)

Requesting URL: https://help.community.com/hc/en-us/
Response Status Code: 403
Reponse  <Response [403]>


In [17]:
from bs4 import BeautifulSoup
import markdownify
import os

# Directory containing the scraped HTML files
html_dir = 'scraped_html'
# Directory to save the markdown files
markdown_dir = 'markdown_files'
os.makedirs(markdown_dir, exist_ok=True)

# Function to convert HTML content to Markdown and save it
def convert_html_to_markdown(html_file):
    with open(html_file, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
        article_content = soup.find('article')
        if article_content:
            markdown_content = markdownify.markdownify(str(article_content), heading_style="ATX")
            markdown_filename = os.path.join(markdown_dir, os.path.basename(html_file).replace('.html', '.md'))
            with open(markdown_filename, 'w', encoding='utf-8') as md_file:
                md_file.write(markdown_content)

# Iterate over all HTML files in the directory and convert them
for html_file in os.listdir(html_dir):
    if html_file.endswith('.html'):
        convert_html_to_markdown(os.path.join(html_dir, html_file))

In [37]:
import os
import openai

# Function to load environment variables from a .env file
def load_env_file(env_file):
    with open(env_file, 'r') as file:
        for line in file:
            if line.strip() and not line.startswith('#'):
                key, value = line.strip().split('=', 1)
                os.environ[key] = value

# Load the .env file
load_env_file('.env')

In [50]:
import os
import openai

# Load the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Directory containing the Markdown files
markdown_dir = 'markdown_files'

# Log file to save file_ids
log_file_path = 'uploaded_files_log.txt'

# Function to upload file to OpenAI
def upload_file_to_openai(file_path, purpose):
    with open(file_path, 'rb') as file:
        response = openai.files.create(
            file=file,
            purpose=purpose
        )
    return response

file_ids_for_vs = []

# Iterate over all Markdown files in the directory and upload
with open(log_file_path, 'a', encoding='utf-8') as log_file:
    for markdown_file in os.listdir(markdown_dir):
        if markdown_file.endswith('.md'):
            file_path = os.path.join(markdown_dir, markdown_file)
            
            # Upload Markdown file to OpenAI
            response = upload_file_to_openai(file_path, 'assistants')
            file_id = response.id
            file_ids_for_vs.append(file_id)
            log_file.write(f"{file_path}: {file_id}\n")
            print(f"Uploaded {file_path}: {file_id}")

print("All files uploaded successfully.")

Uploaded markdown_files/salesforce.md: file-s1yZU3Zc0LVK0NPxuRyEibpf
Uploaded markdown_files/webhook-api-introduction.md: file-KXtny0vXGDJR2uc8qNvnpdsL
Uploaded markdown_files/opt-out-member.md: file-YQN0GEE5EFdnwn48cj5UGKPD
Uploaded markdown_files/send-message.md: file-1VIGb3cvsTv3xaWBcfx3cGVb
Uploaded markdown_files/getting-started-with-your-api.md: file-CHepcwViZSqZedMdcxfPCap4
Uploaded markdown_files/upsert-member.md: file-Ejk8XynV7AkWKvVB4KjfBJ9T
Uploaded markdown_files/webhook-api-authentication.md: file-cGLgzmwxJ8HVlggQEhmbx9R4
Uploaded markdown_files/tag-delete.md: file-76JR6ik62hMHeSxbchArUsPh
Uploaded markdown_files/formstack.md: file-2HowqEchKVQcr68iQveiecjG
Uploaded markdown_files/zapier.md: file-4tDdq8V9FpZQ6F3xOrypQ4x6
Uploaded markdown_files/data-export-introduction.md: file-LdfJotBabhSFiafFPUFqM05x
Uploaded markdown_files/community-member-states.md: file-sHCIuBWOPUrWKmVxnEXxB52y
Uploaded markdown_files/data-export-api-downloading-data.md: file-Wyo48p8TJE6jkkxA9ZmcXlYK
U

In [51]:
import os
import openai

# Load the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Create a vector store using the collected file IDs
vector_store = openai.beta.vector_stores.create(
    name="Community API Reference",
    file_ids=file_ids_for_vs
)

print(vector_store)
print(vector_store.id)

VectorStore(id='vs_jerRxASjrMr8RjWvd9X1Tw67', created_at=1730815163, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=23, total=23), last_active_at=1730815163, metadata={}, name='Community API Reference', object='vector_store', status='in_progress', usage_bytes=0, expires_after=None, expires_at=None)
vs_jerRxASjrMr8RjWvd9X1Tw67


In [60]:
from time import sleep

embeddings_finished = False

while embeddings_finished == False:
    vector_store_status = openai.beta.vector_stores.retrieve(vector_store.id)
    if vector_store_status.status == 'completed':
        print(vector_store_status.status)
        print(f"completed {vector_store_status.file_counts.completed}")
        print(f"in progress {vector_store_status.file_counts.in_progress}")
        print(f"total {vector_store_status.file_counts.total}")
        embeddings_finished = True
        break
    sleep(10)

completed
completed 23
in progress 0
total 23
